In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

In [69]:
df = pd.read_csv('tweets_stocks_combined_5mins.csv')

In [70]:
X = df.loc[:, 'cleaned_text2']
y = df.loc[:, '5mins_price_diff_perc']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [72]:
corpus_list = []

for i in X_train:
    corpus_list.append(i.split(' '))

In [73]:
word2vec_model = Word2Vec(corpus_list, min_count=1, size=100)
pretrained_weights = word2vec_model.wv.vectors

Word2Vec(vocab=8915, size=100, alpha=0.025)


In [74]:
num_words = [len(i) for i in corpus_list]
longest_sentence_len = max(num_words)

In [76]:
def sentence_to_indices_padded(sentences, longest_sentence_len):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [77]:
X_train_padded = sentence_to_indices_padded(X_train, longest_sentence_len)
X_test_padded = sentence_to_indices_padded(X_test, longest_sentence_len)

In [79]:
## Glove index 

embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [80]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5714 unique tokens.


/Users/Amey/anaconda3/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [82]:
w2c = dict()
for item in word2vec_model.wv.vocab:
    w2c[item]=word2vec_model.wv.vocab[item].count
    
w2c_sorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
print(w2c_sorted)

{'the': 2365, 'to': 1346, 'and': 1278, 'of': 1074, '': 1065, 'a': 845, 'in': 752, 'is': 716, 'for': 605, 'on': 487, 'great': 451, 'our': 435, 'will': 422, 'with': 403, 'are': 397, 'we': 382, 'i': 367, 'that': 363, 'be': 352, '&amp,': 324, 'it': 277, 'was': 263, 'have': 260, 'they': 257, 'at': 251, 'you': 249, 'he': 220, 'my': 211, 'has': 208, '-': 207, 'this': 191, 'all': 188, 'by': 187, 'very': 181, 'not': 173, 'who': 162, 'from': 160, 'thank': 156, 'just': 151, 'so': 149, 'people': 148, 'as': 143, 'no': 127, 'many': 125, 'big': 125, 'an': 122, 'his': 118, 'u.s.': 114, 'fake': 114, 'their': 110, 'news': 110, 'america': 109, 'president': 107, 'get': 107, 'about': 103, 'but': 102, 'been': 101, 'or': 97, 'trump': 95, 'new': 94, 'out': 94, 'do': 93, 'there': 92, 'american': 92, 'now': 90, 'much': 89, 'than': 88, 'more': 86, 'make': 86, 'going': 85, 'what': 83, 'tax': 82, 'up': 81, 'democrats': 81, '.': 80, 'your': 80, 'would': 77, 'being': 76, 'if': 76, 'must': 74, 'country': 74, 'want': 

In [91]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [93]:
words_w2v = list(w2c.keys())
words_gloves = list(glove_word_count.keys())
extra = []
for word in words_w2v:
    if word not in words_gloves:
        extra.append(word)
print(extra)    

['trump.”', 'kim.', 'together!', 'indignant,', 'u.s.', 'things...but', '300%', 'farmers,', 'agriculture!', 'east.”', '', 'true,', 'laura!', '....china,', 'us,', 'p.r.', 'tariffs-', 'economy.', 'countries.', 'winning,', 'strong!', 'anthem,', '&amp,', 'country.', 'nfl!', 'be....', '...and', 'leader.', 'security.', '10:00', 'a.m.', '-', 'enjoy!', 'strange,', 'alabama,', '7:15.', 'crime,', 'etc.', '#america', 'first!', 'you!', '#maga', 'truth.', 'joke.', '.', 'sad!', 'july,', '219,000', '185,000', 'estimate:', 'today,', 'border,', 'approval.', 'fantastic.', 'military,', 'amendment.', 'endorsement!', ',', 'all,', 'obama?', 'time!', 'documents.', 'courts.', 'rigged,', 'scam!', 'headline:', 'times.', 'also,', 'may.', 'canada,', 'years!', '45,000', 'region.', '$20', 'investment.', 'again,', 'america!', 'duluth,', 'minnesota.', 'family!', 'house.', 'victory,', 'away.', 'win!', '!', '"""remarks', 'u.s.a.', 'cuba""video:', '....guilty', 'bases.', 'sentence,', 'did-including', 'hook.', 'lawyer,', 

In [90]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)

vocab_size_glove, embedding_size_glove = embedding_matrix.shape
print(vocab_size_glove)
print(embedding_size)
print(count)
print(skipped_words)
embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=longest_sentence_len,
                            trainable=False)

In [50]:
vocab_size, embedding_size = pretrained_weights.shape

In [60]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        if word_to_vec_map[word].shape[0] == emb_dim:
            emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it non-trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = keras.layers.Embedding(input_dim=vocab_len, output_dim=emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [61]:
def create_LSTM_model(pretrained_weights, longest_sentence_len):
    vocab_size, embedding_size = pretrained_weights.shape
    
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights]))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    
    return model

In [62]:
model = create_LSTM_model(pretrained_weights, longest_sentence_len)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 158, 100)          919500    
_________________________________________________________________
LSTM1 (LSTM)                 (None, 158, 4)            1680      
_________________________________________________________________
Dropout1 (Dropout)           (None, 158, 4)            0         
_________________________________________________________________
LSTM2 (LSTM)                 (None, 4)                 144       
_________________________________________________________________
Dropout2 (Dropout)           (None, 4)                 0         
_________________________________________________________________
Dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                

In [63]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
model.fit(X_train_padded, y_train, epochs=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
1788/1788 [==============================] - 23s 13ms/sample - loss: 0.0692 - mean_absolute_error: 0.1773
Epoch 2/50
1788/1788 [==============================] - 22s 12ms/sample - loss: 0.0493 - mean_absolute_error: 0.1453
Epoch 3/50
1788/1788 [==============================] - 23s 13ms/sample - loss: 0.0377 - mean_absolute_error: 0.1282
Epoch 4/50
1788/1788 [==============================] - 26s 14ms/sample - loss: 0.0279 - mean_absolute_error: 0.1094
Epoch 5/50
1788/1788 [==============================] - 29s 16ms/sample - loss: 0.0253 - mean_absolute_error: 0.1059
Epoch 6/50
1788/1788 [==============================] - 36s 20ms/sample - loss: 0.0192 - mean_absolute_error: 0.0916
Epoch 7/50
1788/1788 [==============================] - 24s 13ms/sample - loss: 0.0167 - mean_absolute_error: 0.0879
Epoch 8/50
1788/1788 [==============================] - 21s 12ms/sample - loss: 0.0150 